In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN', 'APPLICATION_TYPE', and 'NAME'.
application_df.drop(columns=["EIN", "NAME", "APPLICATION_TYPE"], inplace=True)

In [3]:
# Confirm column drop
application_df.columns

Index(['AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS',
       'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [8]:
application_df["SPECIAL_CONSIDERATIONS"].replace("N", 0, inplace=True)
application_df["SPECIAL_CONSIDERATIONS"].replace("Y", 1, inplace=True)

In [9]:
# Determine the number of unique values in each column.
application_df.nunique()

AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [10]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_types = application_df.groupby("CLASSIFICATION").count()
classifications_to_replace = classification_types.loc[classification_types['IS_SUCCESSFUL'] < 1800]
classifications_to_replace = list(classifications_to_replace.index.values)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
new_application_df = pd.get_dummies(application_df, dtype=int)

In [14]:
new_application_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [15]:
# Split our preprocessed data into our features and target arrays
features = new_application_df.drop(columns="IS_SUCCESSFUL")
target = new_application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    stratify=target)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()


# First  layer
nn.add(tf.keras.layers.Dense(units=86, activation="relu", input_dim=33))

# Hidden Layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 86)             │         2,924 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 30)             │         2,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,565 (21.74 KB)

 Trainable params: 5,565 (21.74 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Create callback that saves model's weights every five epochs as described in TensorFlow documentation https://www.tensorflow.org/tutorials/keras/save_and_load#:~:text=%23%20Create%20a%20callback%20that%20saves%20the%20model%27s%20weights%20every%205%20epochs%0Acp_callback%20%3D%20tf.keras.callbacks.ModelCheckpoint(%0A%C2%A0%20%C2%A0%20filepath%3Dcheckpoint_path%2C%20%0A%C2%A0%20%C2%A0%20verbose%3D1%2C%20%0A%C2%A0%20%C2%A0%20save_weights_only%3DTrue%2C%0A%C2%A0%20%C2%A0%20save_freq%3D5*n_batches)
file_path = "attempt1.weights.h5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= file_path,
    verbose=1,
    save_weights_only=True,
    save_freq=215)

# Save the weights using the `checkpoint_path` format
nn.save_weights(file_path.format(epoch=0))

In [33]:
# Train the model
nn.fit(X_train_scaled,
       y_train,
       epochs=200,
       callbacks=cp_callback)

Epoch 1/200
203/804 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - accuracy: 0.6625 - loss: 0.6332
Epoch 1: saving model to attempt1.weights.h5
410/804 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.6767 - loss: 0.6176
Epoch 1: saving model to attempt1.weights.h5
613/804 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.6824 - loss: 0.6104
Epoch 1: saving model to attempt1.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step - accuracy: 0.6856 - loss: 0.6064
Epoch 2/200
  1/804 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7188 - loss: 0.6154
Epoch 2: saving model to attempt1.weights.h5
252/804 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.7179 - loss: 0.5667
Epoch 2: saving model to attempt1.weights.h5
452/804 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - accuracy: 0.7147 - loss: 0.5702
Epoch 2: saving model to attempt1.weights.h5
665/804 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.7122 - loss: 0.5729
Epoch 2: saving model to attempt1.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - a

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6442 - accuracy: 0.7332 - 468ms/epoch - 2ms/step
Loss: 0.6442193388938904, Accuracy: 0.7331778407096863


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization2.h5")

This model involved 
* Increasing the numbers of epochs, 
* Increasing the numbers of layers,
* Increasing the number of neurons in the first layer
* Removing application type.